In [34]:
import pandas as pd
import plotly.express as px
import re
from collections import Counter

In [5]:
dataset = pd.read_csv(r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\raw\saudemental.csv')

In [21]:
def cleaning_data(data):
    dataset = data.copy()
    dataset.drop(columns=[
    'conversation_id', 
    'timezone', 
    'time', 
    'place', 
    'mentions', 
    'geo', 
    'source', 
    'user_rt_id', 
    'user_rt', 
    'trans_src',
    'created_at',
    'user_id',
    'name',
    'video',
    'thumbnail',
    'translate',
    'trans_dest',
    'reply_to',
    'source',
    'near',
    'quote_url',
    'retweet',
    'cashtags',
    'hashtags',
    'retweets_count',
    'replies_count',
    'photos',
    'urls',
    'retweet_id',
    'retweet_date',
    'link',
    'likes_count'],
    inplace=True)

    dataset = dataset[dataset['language'] == 'pt']
    dataset.drop_duplicates('tweet', inplace=True)
    dataset.drop_duplicates('username', inplace=True)
    dataset.drop(columns=['language', 'id', 'username'], inplace=True)
    dataset.set_index('date', inplace=True)

    return dataset

In [22]:
#dataset.to_csv(r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\processed\' + data + r'.csv')

In [23]:
data = cleaning_data(dataset)

In [55]:
data['hashtags'] = [", ".join(y for y in re.findall('\B#\w\w+', x)) for x in data['tweet']]

In [57]:
data

,tweet,hashtags
date,,
2021-04-29,#Ansiedade não se resolve com um convite para ...,"#Ansiedade, #ansiedadenaoéfrescura, #Terapia, ..."
2021-04-29,Vamos acompanhar de perto esse processo para g...,"#SiriNaCâmara, #CâmaraMunicipal, #RiodeJaneiro..."
2021-04-29,A pandemia veio e mexeu primeiro com a saúde ...,"#saúde, #COVID19, #SaúdeMental, #mentalhealth,..."
2021-04-29,#saúdemental O Centro de Sáude de #Ordes conta...,"#saúdemental, #Ordes, #Frades, #Mesía, #Tordoi..."
2021-04-29,Esse diálogo da @juliette e o @gilnogueiraofc ...,"#saúdemental, #bbb"
...,...,...
2014-03-26,Denuncie o #assédio moral no trabalho. O silên...,"#assédio, #higieneesegurancadotrabalho, #meioa..."
2014-03-20,O que eu colocar no meu Portifólio de saúde me...,"#AcabaLogo, #saúdeMental"
2014-03-11,"Televisão e Facebook, coisas que não fazem fal...",#saúdemental
